In [ ]:
import numpy as np
import cv2
import os
import csv
tem_path = 'data/template'
arc_path = 'data/archive'#'data/archive'
out_path = 'data/out'
test_path1 = 'data/datatest'
test_out1 = 'data/dataout'

In [ ]:
temlist=[]
for tem in os.listdir(tem_path):
    if tem.endswith('.jpg'):
        img = cv2.imread(os.path.join(tem_path,tem))
        temlist.append(img)

img4 = temlist[0]




In [ ]:
import cv2
import numpy as np
import os

MIN_MATCH_COUNT = 8
name = []
xx1 = []
xx2 = []
yy1 = []
yy2 = []

# 提取img4的ORB特征
gray4 = cv2.cvtColor(img4, cv2.COLOR_BGR2GRAY)
orb = cv2.ORB_create()
kp4, des4 = orb.detectAndCompute(gray4, None)

for arc in os.listdir(arc_path):
    if arc.endswith('.jpg'):
        img1 = cv2.imread(os.path.join(arc_path, arc))
        gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)

        # 定义FLANN匹配器------KD树
        index_params = dict(algorithm=1, trees=5)
        search_params = dict(checks=50)
        flann = cv2.FlannBasedMatcher(index_params, search_params)

        # 使用ORB提取特征
        kp1, des1 = orb.detectAndCompute(gray1, None)

        # 将特征描述子转换为np.float32类型
        des1 = np.float32(des1)
        des4 = np.float32(des4)

        matches = flann.knnMatch(des4, des1, k=2)

        # 去除错误匹配
        good = []
        for m, n in matches:
            if m.distance <= 0.8 * n.distance:
                good.append(m)

        if len(good) > MIN_MATCH_COUNT:
            # 改变数组的表现形式，不改变数据内容，数据内容是每个关键点的坐标位置
            src_pts = np.float32([kp4[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
            dst_pts = np.float32([kp1[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)

            # findHomography 函数是计算变换矩阵
            # 参数cv2.RANSAC是使用RANSAC算法寻找一个最佳单应性矩阵H，即返回值M
            # 返回值：M 为变换矩阵，mask是掩模
            M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
            matchesMask = mask.ravel().tolist()

            if sum(matchesMask) > MIN_MATCH_COUNT:
                h, w = img4.shape[:2]
                pts = np.float32([[0, 0], [0, h - 1], [w - 1, h - 1], [w - 1, 0]]).reshape(-1, 1, 2)
                dst = cv2.perspectiveTransform(pts, M)
                dst = np.int32(dst)

                x1, y1 = dst[0][0]
                x2, y2 = dst[2][0]
                name.append(arc)
                xx1.append(x1)
                yy1.append(y1)
                xx2.append(x2)
                yy2.append(y2)

                img1 = cv2.polylines(img1, [dst], True, (255, 255, 255), 3, cv2.LINE_AA)
                draw_params = dict(matchColor=(0, 255, 0), singlePointColor=None, matchesMask=matchesMask, flags=2)
                img5 = cv2.drawMatches(img4, kp4, img1, kp1, good, None, **draw_params)
                cv2.imwrite(os.path.join(out_path, arc), img5)
            else:
                print("Not enough inliers are found.")
        else:
            print("Not enough matches are found.")


In [ ]:
print(name,xx1,yy1,xx2,yy2)

In [ ]:
MIN_MATCH_COUNT = 8
name = []
xx1 = []
xx2 = []
yy1 = []
yy2 = []

for arc in os.listdir(arc_path):
    if arc.endswith('.jpg'):
        img1 = cv2.equalizeHist(cv2.cvtColor(cv2.imread(os.path.join(arc_path, arc)), cv2.COLOR_BGR2GRAY))

        # 定义FLANN匹配器------KD树
        index_params = dict(algorithm=1, trees=5)
        search_params = dict(checks=50)
        flann = cv2.FlannBasedMatcher(index_params, search_params)

        sift = cv2.SIFT_create()  # 使用SIFT特征提取器
        kp1, des1 = sift.detectAndCompute(img1, None)

        # 匹配图像
        matches = flann.knnMatch(des4, des1, k=2)
        good = []
        for m, n in matches:
            if m.distance < 0.7 * n.distance:
                good.append(m)

        if len(good) > MIN_MATCH_COUNT:
            src_pts = np.float32([kp4[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
            dst_pts = np.float32([kp1[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)
            M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

            matchesMask = mask.ravel().tolist()
            h, w, dim = img4.shape
            pts = np.float32([[0, 0], [0, h - 1], [w - 1, h - 1], [w - 1, 0]]).reshape(-1, 1, 2)
            dst = cv2.perspectiveTransform(pts, M)
            dst = np.int32(dst)
            x1, y1 = dst[0][0]
            x2, y2 = dst[2][0]
            name.append(arc)
            xx1.append(x1)
            yy1.append(y1)
            xx2.append(x2)
            yy2.append(y2)
            img1 = cv2.polylines(img1, [np.int32(dst)], True, (255, 255, 255), 3, cv2.LINE_AA)
            draw_params = dict(matchColor=(0, 255, 0), singlePointColor=None, matchesMask=matchesMask, flags=2)
            img5 = cv2.drawMatches(img4, kp4, img1, kp1, good, None, **draw_params)
            cv2.imwrite(os.path.join(out_path, arc), img5)
        else:
            print("Not enough matches are found - %d/%d" % (len(good), MIN_MATCH_COUNT))


In [ ]:
import pandas as pd

df= pd.DataFrame({'archive_jpg':name,'x1':xx1,'y1':yy1,'x2':xx2,'y2':yy2})
df.to_csv("out.csv",index=False)